In [12]:
import random
import numpy as np
import time

class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[] for _ in range(vertices)]
    
    def add_edge(self, u, v):
        if v not in self.graph[u] and u != v:
            self.graph[u].append(v)
            self.graph[v].append(u)
    
    def print_graph(self):
        for i in range(self.V):
            print(f"{i} --> {' '.join(map(str, self.graph[i]))}")

def generate_uniform(graph, E):
    added_edges = set()
    while E > 0:
        u, v = random.randint(0, graph.V - 1), random.randint(0, graph.V - 1)
        if u != v and (u, v) not in added_edges and (v, u) not in added_edges:
            graph.add_edge(u, v)
            added_edges.add((u, v))
            E -= 1

def generate_skewed(graph, E):
    added_edges = set()
    while E > 0:
        u, v = int(graph.V * (1 - np.sqrt(1 - random.uniform(0, 1)))), int(graph.V * (1 - np.sqrt(1 - random.uniform(0, 1))))
        if u != v and (u, v) not in added_edges and (v, u) not in added_edges:
            graph.add_edge(u, v)
            added_edges.add((u, v))
            E -= 1

def generate_cycle_graph(graph):
    # Generates a cycle graph
    for i in range(graph.V):
        if (i == graph.V - 1):  # this is the last node, wrap around to start
            graph.add_edge(i, 0)
        else:  # This is a non-terminal node, so just connect to adjacent node
            graph.add_edge(i, i + 1)

def generate_complete_graph(graph):
    # Generates a complete graph
    for i in range(graph.V):
        for j in range(i + 1, graph.V):
            graph.add_edge(i, j)

def save_graph_to_file(graph, filename):
    with open(filename, 'w') as file:
        for u in range(graph.V):
            file.write(f"{u}: {' '.join(map(str, graph.graph[u]))}\n")

def main(V, G, filename, DIST='UNIFORM'):
    elapsed_time_microseconds = 0
    graph = Graph(V)
    if G == 'RANDOM':
        if DIST == 'UNIFORM':
            print('Graph Type: Random Uniform Distrib')
            start_time = time.perf_counter()
            generate_uniform(graph, int(V*(V-1)/4))  # Example for medium density
            elapsed_time = time.perf_counter() - start_time
            elapsed_time_microseconds = elapsed_time * 1_000_000
            
        elif DIST == 'SKEWED':
            print('Graph Type: Random Skewed Distrib')
            start_time = time.perf_counter()
            generate_skewed(graph, int(V*(V-1)/4))  # Example for medium density
            elapsed_time = time.perf_counter() - start_time
            elapsed_time_microseconds = elapsed_time * 1_000_000
    elif G == 'CYCLE':
        print('Graph Type: Cycle')
        start_time = time.perf_counter()
        generate_cycle_graph(graph)
        elapsed_time = time.perf_counter() - start_time
        elapsed_time_microseconds = elapsed_time * 1_000_000
    elif G == 'COMPLETE':
        print('Graph Type: Complete')
        start_time = time.perf_counter()
        generate_complete_graph(graph)
        elapsed_time = time.perf_counter() - start_time
        elapsed_time_microseconds = elapsed_time * 1_000_000
        
    # graph.print_graph()
    print(f"Elapsed Time: {elapsed_time_microseconds:.2f} microseconds\n")
    save_graph_to_file(graph, filename)


# Example Usage
for v in range(10, 101, 10):
    print("Num Verts: ", v)
    main(V=v, G='RANDOM', filename="output_random_uniform.txt")
    
for v in range(10, 101, 10):
    print("Num Verts: ", v)
    main(V=v, G='RANDOM', DIST='SKEWED', filename="output_random_uniform.txt")

for v in range(10, 101, 10):
    print("Num Verts: ", v)
    main(V=v, G='CYCLE', filename="output_cycle_graph.txt")
#     
for v in range(10, 101, 10):
    print("Num Verts: ", v)
    main(V=v, G='COMPLETE', filename="output_complete_graph.txt")
    v += 10

Num Verts:  10
Graph Type: Random Uniform Distrib
Elapsed Time: 69.80 microseconds

Num Verts:  20
Graph Type: Random Uniform Distrib
Elapsed Time: 202.60 microseconds

Num Verts:  30
Graph Type: Random Uniform Distrib
Elapsed Time: 409.70 microseconds

Num Verts:  40
Graph Type: Random Uniform Distrib
Elapsed Time: 855.90 microseconds

Num Verts:  50
Graph Type: Random Uniform Distrib
Elapsed Time: 1177.80 microseconds

Num Verts:  60
Graph Type: Random Uniform Distrib
Elapsed Time: 1768.90 microseconds

Num Verts:  70
Graph Type: Random Uniform Distrib
Elapsed Time: 2455.40 microseconds

Num Verts:  80
Graph Type: Random Uniform Distrib
Elapsed Time: 2686.90 microseconds

Num Verts:  90
Graph Type: Random Uniform Distrib
Elapsed Time: 3564.80 microseconds

Num Verts:  100
Graph Type: Random Uniform Distrib
Elapsed Time: 4274.10 microseconds

Num Verts:  10
Graph Type: Random Skewed Distrib
Elapsed Time: 188.50 microseconds

Num Verts:  20
Graph Type: Random Skewed Distrib
Elapsed Tim

In [3]:
import random
import time

class GraphColoring:
    def __init__(self, filename):
        self.graph = self.read_graph(filename)
        self.V = len(self.graph)
        self.colors = [-1] * self.V
        self.degrees = [len(adj) for adj in self.graph]

    def read_graph(self, filename):
        graph = []
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split(':')
                if len(parts) == 2:
                    _, edges = parts
                    graph.append(list(map(int, edges.split())))
                else:
                    graph.append([])
        return graph

    def smallest_last_ordering(self):
        ordering = []
        degrees = self.degrees.copy()
        for _ in range(self.V):
            min_degree_vertex = degrees.index(min(degrees))
            ordering.append(min_degree_vertex)
            degrees[min_degree_vertex] = self.V + 1
            for neighbor in self.graph[min_degree_vertex]:
                degrees[neighbor] -= 1
        return ordering[::-1]

    def smallest_original_degree_last(self):
        return sorted(range(self.V), key=lambda x: self.degrees[x])

    def uniform_random_ordering(self):
        ordering = list(range(self.V))
        random.shuffle(ordering)
        return ordering

    def largest_degree_first_ordering(self):
        return sorted(range(self.V), key=lambda x: -self.degrees[x])

    def degree_of_saturation_ordering(self):
        saturation = [0] * self.V
        ordering = []
        while len(ordering) < self.V:
            max_sat = -1
            for i in range(self.V):
                if i not in ordering and (saturation[i] > max_sat or (saturation[i] == max_sat and self.degrees[i] > self.degrees[vertex])):
                    max_sat = saturation[i]
                    vertex = i
            ordering.append(vertex)
            for neighbor in self.graph[vertex]:
                if neighbor not in ordering:
                    saturation[neighbor] += 1
        return ordering

    def color_graph(self, ordering):
        for vertex in ordering:
            forbidden = [False] * self.V
            for neighbor in self.graph[vertex]:
                if self.colors[neighbor] != -1:
                    forbidden[self.colors[neighbor]] = True
            self.colors[vertex] = next(color for color, used in enumerate(forbidden) if not used)

    def print_coloring_results(self):
        for i, color in enumerate(self.colors):
            print(f"Vertex {i}: Color {color}")

    def process_graph(self, filename):
        print(f"\nProcessing file: {filename}")
        orderings = [
            (self.smallest_last_ordering, "Smallest Last Ordering"),
            (self.smallest_original_degree_last, "Smallest Original Degree Last"),
            (self.uniform_random_ordering, "Uniform Random Ordering"),
            (self.largest_degree_first_ordering, "Largest Degree First Ordering"),
            (self.degree_of_saturation_ordering, "Degree of Saturation Ordering"),
        ]
        for method, name in orderings:
            start_time = time.perf_counter()
            print(f"\n{name}:")
            self.color_graph(method())
            self.print_coloring_results()
            elapsed_time = time.perf_counter() - start_time
            elapsed_time_microseconds = elapsed_time * 1_000_000
            print(f"Elapsed Time: {elapsed_time_microseconds:.2f} microseconds")
            print(f"Total Colors Used: {max(self.colors) + 1}")
            self.colors = [-1] * self.V

def main():
    filenames = ["output_complete_graph.txt", "output_random_uniform.txt", "output_cycle_graph.txt"]
    for filename in filenames:
        graph_coloring = GraphColoring(filename)
        graph_coloring.process_graph(filename)

if __name__ == "__main__":
    main()



Processing file: output_complete_graph.txt

Smallest Last Ordering:
Vertex 0: Color 9
Vertex 1: Color 8
Vertex 2: Color 7
Vertex 3: Color 6
Vertex 4: Color 5
Vertex 5: Color 4
Vertex 6: Color 3
Vertex 7: Color 2
Vertex 8: Color 1
Vertex 9: Color 0
Elapsed Time: 62.10 microseconds
Total Colors Used: 10

Smallest Original Degree Last:
Vertex 0: Color 0
Vertex 1: Color 1
Vertex 2: Color 2
Vertex 3: Color 3
Vertex 4: Color 4
Vertex 5: Color 5
Vertex 6: Color 6
Vertex 7: Color 7
Vertex 8: Color 8
Vertex 9: Color 9
Elapsed Time: 40.30 microseconds
Total Colors Used: 10

Uniform Random Ordering:
Vertex 0: Color 5
Vertex 1: Color 7
Vertex 2: Color 2
Vertex 3: Color 4
Vertex 4: Color 0
Vertex 5: Color 6
Vertex 6: Color 9
Vertex 7: Color 8
Vertex 8: Color 1
Vertex 9: Color 3
Elapsed Time: 46.30 microseconds
Total Colors Used: 10

Largest Degree First Ordering:
Vertex 0: Color 0
Vertex 1: Color 1
Vertex 2: Color 2
Vertex 3: Color 3
Vertex 4: Color 4
Vertex 5: Color 5
Vertex 6: Color 6
Vertex 7: 